In [2]:

crimeData = pd.read_feather("../src/data/updatedcrimedata")
crimeData

,index,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_LON,GEO_LAT,DISTRICT_ID,IS_CRIME,IS_TRAFFIC,COLOR
0,351573,20156000026,20156000026230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,2015-01-02 00:00:00,2015-01-02 12:00:00,2015-01-02 13:47:00,4904 N JULIAN ST,-105.030728,39.785783,1,1,0,green
1,238911,201513644,201513644230800,2308,0,theft-from-bldg,larceny,2015-01-02 00:00:00,2015-01-08 09:20:00,2015-01-08 10:53:00,6750 E STAPLETON S DR,-104.910120,39.777321,2,1,0,green
2,87955,20156000023,20156000023230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,2015-01-02 00:00:00,2015-01-02 07:00:00,2015-01-02 13:21:00,2121 DELGANY ST,-104.997312,39.757979,6,1,0,green
3,351572,20156000022,20156000022230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,2015-01-02 00:00:00,2015-01-02 07:00:00,2015-01-02 12:00:00,4455 N WOLFF ST,-105.048907,39.777578,1,1,0,green
4,65429,20151811,20151811549900,5499,0,traf-other,all-other-crimes,2015-01-02 01:03:00,2015-01-02 01:03:00,2015-01-02 03:27:00,N CENTRAL PARK BLVD / E NORTHFIELD BLVD,-104.883070,39.785728,5,1,0,all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456952,60089,202045112,202045112549900,5499,0,traf-other,all-other-crimes,2020-01-21 03:33:00,2020-01-21 03:33:00,2020-01-21 04:39:00,E 4TH AVE / N GRANT ST,-104.983804,39.722545,3,1,0,all
456953,60078,202045132,202045132739902,7399,2,public-order-crimes-other,all-other-crimes,2020-01-21 03:56:00,2020-01-21 03:56:00,2020-01-21 03:56:00,100 W COLFAX AVE,-104.988833,39.740354,6,1,0,all
456954,60082,202045137,202045137549900,5499,0,traf-other,all-other-crimes,2020-01-21 03:59:00,2020-01-21 03:59:00,2020-01-21 04:34:00,4400 BLOCK MORRISON RD,-105.043983,39.703058,4,1,0,all
456955,60062,202045149,202045149739902,7399,2,public-order-crimes-other,all-other-crimes,2020-01-21 04:17:00,2020-01-21 04:17:00,2020-01-21 04:17:00,1499 N BROADWAY ST,-104.987485,39.739897,6,1,0,all


In [47]:
import sys
sys.path.append('../src/')

from data.CrimeDataManager import CrimeDataManager
from live.ScoreCore import ScoreCore

import pandas as pd
from datetime import datetime
from dateutil import parser
import ast


In [50]:
class Pin:
    def __init__(self, pinId, code, lat, lon, color, timePlaced,  userPlaced):
        self.pinId = pinId
        self.code = code.encode()
        self.lat = lat
        self.lon = lon
        self.color = color.encode()
        self.timePlaced = timePlaced
        self.userPlaced = userPlaced.encode()
        self.userMoved = userPlaced.encode()

    def toJSON(self):
        return {
            'pinId': self.pinId,
            'code': self.code,
            'lat': self.lat,
            'lon': self.lon,
            'color': self.color,
            'timePlaced': self.timePlaced.strftime('%Y-%m-%d %H:%M:%S.%f')
        }

    def __str__(self):
        return self.toJSON().__str__()

    

def groupToPins(group):
    pins = []
    for _, row in group.iterrows():
        time = datetime.strptime(row['serverTime'], '%Y-%m-%d %H:%M:%S.%f')
        pin = Pin(pinId=row['pinId'], code=row['code'], lat=row['pinLat'], lon=row['pinLon'], color=row['pinColor'], timePlaced=time, userPlaced=row['primaryUserId'])
        pins.append(pin)
    return pins

def generatePinListFromCSV(csvPath):
    # load CSV
    csv_df = pd.read_csv(csvPath)


    # we only pay attention to the last event for the pin in the round
    lasts_df = csv_df.drop_duplicates(subset=['pinId', 'roundNumber'], keep='last')

    # drop any pinDeleted
    movpla_df = lasts_df[lasts_df['eventName'].isin(['pinPlaced', 'pinMoved'])]


    final_df = movpla_df

    display(final_df[final_df['roundNumber'] == 1])


    pin_data = final_df.groupby('roundNumber').apply(groupToPins).to_dict()

    return [[pin for pin in pin_data[key]] for key in pin_data.keys()]

    display(csv_df)

test = generatePinListFromCSV("results-sample.csv")

,Unnamed: 0,code,serverTime,roundNumber,eventName,primaryUserId,secondaryUserId,pinId,pinLat,pinLon,pinColor
1,1,sample,2023-09-06 19:50:24.347431,1,pinPlaced,1,NaN,0.0,39.749445,-105.006981,green
2,2,sample,2023-09-06 19:50:24.773574,1,pinPlaced,1,NaN,1.0,39.728988,-105.042686,green
3,3,sample,2023-09-06 19:50:25.125978,1,pinPlaced,1,NaN,2.0,39.717239,-104.989300,green
4,4,sample,2023-09-06 19:50:26.303599,1,pinPlaced,1,NaN,3.0,39.730440,-104.946556,red
5,5,sample,2023-09-06 19:50:26.743253,1,pinPlaced,1,NaN,4.0,39.744958,-104.977455,red
6,6,sample,2023-09-06 19:50:27.130737,1,pinPlaced,1,NaN,5.0,39.722784,-105.000114,red
7,7,sample,2023-09-06 19:50:27.958747,1,pinPlaced,1,NaN,6.0,39.735324,-105.022259,red
8,8,sample,2023-09-06 19:50:28.641992,1,pinPlaced,2,NaN,7.0,39.745342,-105.056096,green
9,9,sample,2023-09-06 19:50:30.306929,1,pinPlaced,2,NaN,8.0,39.732670,-104.980410,red
10,10,sample,2023-09-06 19:50:30.718184,1,pinPlaced,2,NaN,9.0,39.738214,-104.998945,red


In [ ]:
def fmtDate(dateStr):
    date_obj = parser.parse(dateStr)
    return date_obj.strftime("%Y-%m-%d")

def fmtLayers(layerStr):
    return ast.literal_eval(layerStr)

def getConfigData(filepath):
    cfg_df = pd.read_csv(filepath)

    # get unique round by firsts
    rds_df = cfg_df.drop_duplicates(subset=['Round Num/ID (Int)'], keep='last')

    return [{"layers":fmtLayers(row["Layers (JSON)"]), "date":fmtDate(row["Target Date (Date)"])} for _,row in rds_df.iterrows()]

cfg= getConfigData(filepath="../samples/sample_experiment_config.csv")
display(cfg)

In [62]:
cdm= CrimeDataManager(filepath='../src/data/updatedcrimedata', skipFilepathFormat=True)
sc = ScoreCore(cdm)

def divinateScores(pins, cfg):
    scores = []

    if(len(pins) != len(cfg)):
        print("Your pins and config are off")
        return

    for i in range(len(pins)):
        res = sc.calculateScores(pins=pins[i], date=cfg[i]["date"], categories=cfg[i]["layers"])
        scores.append(res[0])
    
    return scores

scrs = divinateScores(test, cfg)

usr_scrs = [[],[]]

for scr in scrs:
    usr_scrs[0].append(scr['1'])
    usr_scrs[1].append(scr['2'])

display(usr_scrs)

for i in range(len(usr_scrs)):
    print("User " + (i+1).__str__())
    for j in range(len(usr_scrs[i])):
        print(f"Round {j+1}: {usr_scrs[i][j]}")
    print("")


Your pins and config are off


TypeError: 'NoneType' object is not iterable

In [81]:
listOfInputs = ["G01.csv", "G02.csv", "G03.csv", "G04.csv", "G05.csv", "G06.csv", "G07.csv", "G08.csv", "G09.csv", "G10.csv", "G11.csv", "G12.csv", "G13.csv", "G14.csv", "G15.csv", "G16.csv", "G17.csv", "G18.csv", "G19.csv", "G20.csv", "G21.csv", "G22.csv", "G23.csv"]
cfg = getConfigData('hatt2_config.csv')

# listOfInputs = ["results-sample.csv"]
# cfg = getConfigData("../samples/sample_experiment_config.csv")
def getAllScores(inputList):
    scoresPerExperiment = []
    for inputPath in inputList:
        pinList = generatePinListFromCSV(inputPath)
        scores = divinateScores(pinList, cfg)
        scoresPerExperiment.append({"input": inputPath, "scores": scores})
    return scoresPerExperiment

all_scrs = getAllScores(listOfInputs)
        

,Unnamed: 0,code,serverTime,roundNumber,eventName,primaryUserId,secondaryUserId,pinId,pinLat,pinLon,pinColor
1,1,exp1,2023-06-22 15:31:14.299751,1,pinPlaced,AI,NaN,0.0,39.751827,-104.996040,green
2,2,exp1,2023-06-22 15:31:49.298549,1,pinPlaced,AI,NaN,1.0,39.763586,-104.979510,red
3,3,exp1,2023-06-22 15:32:34.311533,1,pinPlaced,AI,NaN,2.0,39.745320,-104.989830,green
10,10,exp1,2023-06-22 15:33:10.057549,1,pinPlaced,1,NaN,5.0,39.748246,-104.990459,red
12,12,exp1,2023-06-22 15:33:13.713808,1,pinMoved,1,1,4.0,39.739270,-104.988612,red
15,15,exp1,2023-06-22 15:33:16.641152,1,pinMoved,1,AI,7.0,39.755769,-104.991706,red
21,21,exp1,2023-06-22 15:33:33.712293,1,pinMoved,2,2,9.0,39.747850,-104.962770,green
23,23,exp1,2023-06-22 15:33:50.231765,1,pinMoved,2,2,6.0,39.739699,-104.974278,green
24,24,exp1,2023-06-22 15:34:29.299884,1,pinPlaced,AI,NaN,10.0,39.689340,-104.959350,green
26,26,exp1,2023-06-22 15:34:31.154926,1,pinMoved,2,2,11.0,39.748576,-104.974793,green


HERO- 2019-05-05 2019-05-05 ['traffic-accident', 'auto-theft', 'robbery', 'murder', 'arson']
         index  INCIDENT_ID        OFFENSE_ID  OFFENSE_CODE  \
393146  202303   2019281641  2019281641540100          5401   
393261  352790   2019282613  2019282613544100          5441   
393200  198507   2019282190  2019282190542000          5420   
393084  208984   2019280935  2019280935544100          5441   
393087  206768   2019282306  2019282306240400          2404   
...        ...          ...               ...           ...   
393167  356033   2019281795  2019281795544100          5441   
393176  206771   2019282061  2019282061540100          5401   
393118  208237   2019281273  2019281273544100          5441   
393264  352031   2019283749  2019283749240400          2404   
393108  355648   2019281198  2019281198544100          5441   

        OFFENSE_CODE_EXTENSION               OFFENSE_TYPE_ID  \
393146                       0  traffic-accident-hit-and-run   
393261                

,Unnamed: 0,code,serverTime,roundNumber,eventName,primaryUserId,secondaryUserId,pinId,pinLat,pinLon,pinColor
1,1,exp1,2023-07-13 10:35:55.901247,1,pinPlaced,AI,NaN,0.0,39.751827,-104.996040,green
3,3,exp1,2023-07-13 10:37:15.901927,1,pinPlaced,AI,NaN,2.0,39.745320,-104.989830,green
6,6,exp1,2023-07-13 10:37:47.041688,1,pinPlaced,1,NaN,6.0,39.754053,-104.994081,red
8,8,exp1,2023-07-13 10:37:58.780092,1,pinPlaced,1,NaN,8.0,39.745540,-104.998804,red
11,11,exp1,2023-07-13 10:38:42.345359,1,pinMoved,1,1,3.0,39.734518,-104.989556,green
12,12,exp1,2023-07-13 10:38:46.488796,1,pinPlaced,2,NaN,9.0,39.764328,-104.972669,green
13,13,exp1,2023-07-13 10:39:00.721572,1,pinMoved,1,AI,7.0,39.738412,-104.988786,red
14,14,exp1,2023-07-13 10:39:10.247877,1,pinPlaced,1,NaN,10.0,39.734452,-104.976736,red
15,15,exp1,2023-07-13 10:39:10.939842,1,pinPlaced,AI,NaN,11.0,39.689340,-104.959350,green
16,16,exp1,2023-07-13 10:39:12.860113,1,pinPlaced,2,NaN,12.0,39.753233,-104.986396,green


HERO- 2019-05-05 2019-05-05 ['traffic-accident', 'auto-theft', 'robbery', 'murder', 'arson']
         index  INCIDENT_ID        OFFENSE_ID  OFFENSE_CODE  \
393146  202303   2019281641  2019281641540100          5401   
393261  352790   2019282613  2019282613544100          5441   
393200  198507   2019282190  2019282190542000          5420   
393084  208984   2019280935  2019280935544100          5441   
393087  206768   2019282306  2019282306240400          2404   
...        ...          ...               ...           ...   
393167  356033   2019281795  2019281795544100          5441   
393176  206771   2019282061  2019282061540100          5401   
393118  208237   2019281273  2019281273544100          5441   
393264  352031   2019283749  2019283749240400          2404   
393108  355648   2019281198  2019281198544100          5441   

        OFFENSE_CODE_EXTENSION               OFFENSE_TYPE_ID  \
393146                       0  traffic-accident-hit-and-run   
393261                

,Unnamed: 0,code,serverTime,roundNumber,eventName,primaryUserId,secondaryUserId,pinId,pinLat,pinLon,pinColor
1,1,exp1,2023-07-13 14:08:50.533416,1,pinPlaced,AI,NaN,0.0,39.751827,-104.996040,green
2,2,exp1,2023-07-13 14:09:17.491776,1,pinPlaced,AI,NaN,1.0,39.763586,-104.979510,red
3,3,exp1,2023-07-13 14:09:56.699725,1,pinPlaced,2,NaN,2.0,39.738984,-104.986882,green
4,4,exp1,2023-07-13 14:10:02.492325,1,pinPlaced,AI,NaN,3.0,39.745320,-104.989830,green
5,5,exp1,2023-07-13 14:10:02.614557,1,pinPlaced,2,NaN,4.0,39.677689,-105.022421,green
6,6,exp1,2023-07-13 14:10:42.520737,1,pinPlaced,AI,NaN,5.0,39.737305,-104.985940,red
7,7,exp1,2023-07-13 14:11:24.080787,1,pinPlaced,1,NaN,6.0,39.761839,-104.973324,green
9,9,exp1,2023-07-13 14:11:46.122823,1,pinPlaced,1,NaN,8.0,39.762763,-105.006507,red
10,10,exp1,2023-07-13 14:11:47.173487,1,pinPlaced,2,NaN,9.0,39.764068,-104.957523,red
12,12,exp1,2023-07-13 14:11:54.219798,1,pinPlaced,2,NaN,10.0,39.711514,-105.022077,red


...

HERO- 2019-05-05 2019-05-05 ['traffic-accident', 'auto-theft', 'robbery', 'murder', 'arson']
         index  INCIDENT_ID        OFFENSE_ID  OFFENSE_CODE  \
393146  202303   2019281641  2019281641540100          5401   
393261  352790   2019282613  2019282613544100          5441   
393200  198507   2019282190  2019282190542000          5420   
393084  208984   2019280935  2019280935544100          5441   
393087  206768   2019282306  2019282306240400          2404   
...        ...          ...               ...           ...   
393167  356033   2019281795  2019281795544100          5441   
393176  206771   2019282061  2019282061540100          5401   
393118  208237   2019281273  2019281273544100          5441   
393264  352031   2019283749  2019283749240400          2404   
393108  355648   2019281198  2019281198544100          5441   

        OFFENSE_CODE_EXTENSION               OFFENSE_TYPE_ID  \
393146                       0  traffic-accident-hit-and-run   
393261                

,Unnamed: 0,code,serverTime,roundNumber,eventName,primaryUserId,secondaryUserId,pinId,pinLat,pinLon,pinColor
1,1,exp1,2023-08-27 10:19:22.872260,1,pinPlaced,AI,NaN,0.0,39.751827,-104.996040,green
2,2,exp1,2023-08-27 10:19:57.876862,1,pinPlaced,AI,NaN,1.0,39.763586,-104.979510,red
4,4,exp1,2023-08-27 10:21:06.262565,1,pinMoved,2,AI,2.0,39.744088,-104.991789,green
5,5,exp1,2023-08-27 10:21:11.961187,1,pinPlaced,1,NaN,3.0,39.744225,-104.993022,red
7,7,exp1,2023-08-27 10:21:22.864203,1,pinPlaced,AI,NaN,5.0,39.737305,-104.985940,red
8,8,exp1,2023-08-27 10:22:03.798038,1,pinPlaced,1,NaN,6.0,39.727858,-104.991948,red
12,12,exp1,2023-08-27 10:22:37.658802,1,pinMoved,2,2,7.0,39.740326,-104.979259,green
13,13,exp1,2023-08-27 10:22:37.860530,1,pinPlaced,AI,NaN,8.0,39.689340,-104.959350,green
15,15,exp1,2023-08-27 10:22:59.309261,1,pinMoved,1,1,9.0,39.753393,-104.986468,red
16,16,exp1,2023-08-27 10:23:04.572567,1,pinMoved,1,1,4.0,39.752338,-104.999943,red


HERO- 2019-05-05 2019-05-05 ['traffic-accident', 'auto-theft', 'robbery', 'murder', 'arson']
         index  INCIDENT_ID        OFFENSE_ID  OFFENSE_CODE  \
393146  202303   2019281641  2019281641540100          5401   
393261  352790   2019282613  2019282613544100          5441   
393200  198507   2019282190  2019282190542000          5420   
393084  208984   2019280935  2019280935544100          5441   
393087  206768   2019282306  2019282306240400          2404   
...        ...          ...               ...           ...   
393167  356033   2019281795  2019281795544100          5441   
393176  206771   2019282061  2019282061540100          5401   
393118  208237   2019281273  2019281273544100          5441   
393264  352031   2019283749  2019283749240400          2404   
393108  355648   2019281198  2019281198544100          5441   

        OFFENSE_CODE_EXTENSION               OFFENSE_TYPE_ID  \
393146                       0  traffic-accident-hit-and-run   
393261                

,Unnamed: 0,code,serverTime,roundNumber,eventName,primaryUserId,secondaryUserId,pinId,pinLat,pinLon,pinColor
1,1,exp1,2023-08-27 13:24:52.470530,1,pinPlaced,AI,NaN,0.0,39.751827,-104.996040,green
12,12,exp1,2023-08-27 13:25:27.469099,1,pinPlaced,AI,NaN,4.0,39.763586,-104.979510,red
18,18,exp1,2023-08-27 13:26:07.192638,1,pinPlaced,1,NaN,5.0,39.763702,-105.009353,green
20,20,exp1,2023-08-27 13:26:12.472539,1,pinPlaced,AI,NaN,6.0,39.745320,-104.989830,green
24,24,exp1,2023-08-27 13:26:48.576740,1,pinPlaced,1,NaN,7.0,39.745100,-105.018972,red
28,28,exp1,2023-08-27 13:26:52.473063,1,pinPlaced,AI,NaN,10.0,39.737305,-104.985940,red
34,34,exp1,2023-08-27 13:27:16.317229,1,pinMoved,1,1,11.0,39.777271,-104.827563,red
36,36,exp1,2023-08-27 13:27:32.013431,1,pinMoved,1,2,8.0,39.714712,-105.040115,red
39,39,exp1,2023-08-27 13:27:47.005449,1,pinMoved,2,2,3.0,39.759200,-104.971619,green
40,40,exp1,2023-08-27 13:27:49.901291,1,pinMoved,1,1,9.0,39.721844,-105.023457,red


HERO- 2019-05-05 2019-05-05 ['traffic-accident', 'auto-theft', 'robbery', 'murder', 'arson']
         index  INCIDENT_ID        OFFENSE_ID  OFFENSE_CODE  \
393146  202303   2019281641  2019281641540100          5401   
393261  352790   2019282613  2019282613544100          5441   
393200  198507   2019282190  2019282190542000          5420   
393084  208984   2019280935  2019280935544100          5441   
393087  206768   2019282306  2019282306240400          2404   
...        ...          ...               ...           ...   
393167  356033   2019281795  2019281795544100          5441   
393176  206771   2019282061  2019282061540100          5401   
393118  208237   2019281273  2019281273544100          5441   
393264  352031   2019283749  2019283749240400          2404   
393108  355648   2019281198  2019281198544100          5441   

        OFFENSE_CODE_EXTENSION               OFFENSE_TYPE_ID  \
393146                       0  traffic-accident-hit-and-run   
393261                

In [82]:
def formatScoresToCsv(scrs):
    rows = []
    for exp in scrs:
        for round_id in range(len(exp['scores'])):
            print(exp['scores'][round_id])
            for usr in exp['scores'][round_id]:
                rows.append({
                    "Experiment": exp['input'],
                    "Round": round_id+1,
                    "User": usr,
                    "Score": exp['scores'][round_id][usr]
                })
    df = pd.DataFrame(rows)
    df.to_csv('scoresG01toG23.csv')

formatScoresToCsv(all_scrs)

{'AI': 0, '1': 4, '2': 4}
{'2': 4, '1': 2, 'AI': 1}
{'2': 6, '1': 1, 'AI': 0}
{'2': 9, '1': 1}
{'2': 5, '1': 1}
{'2': 4, '1': 0, 'AI': 0}
{'2': 9, '1': 1}
{'2': 3, '1': 0}
{'AI': 0, '1': 4, '2': 2}
{'1': 2, '2': 1, 'AI': 1}
{'AI': 1, '2': 3, '1': 1}
{'1': 7, '2': 0, 'AI': 1}
{'1': 4, '2': 0, 'AI': 2}
{'1': 0, '2': 0, 'AI': 0}
{'1': 3, 'AI': 0, '2': 2}
{'1': 1, 'AI': 1, '2': 2}
{'AI': 1, '2': 1, '1': 0}
{'2': 4, 'AI': 2, '1': 0}
{'AI': 2, '2': 7, '1': 3}
{'2': 3, 'AI': 3, '1': 0}
{'AI': 4, '2': 1, '1': 1}
{'1': 0, 'AI': 5, '2': 2}
{'1': 1, 'AI': 2, '2': 8}
{'AI': 1, '1': 0, '2': 1}
{'AI': 1, '1': 0}
{'2': 2, 'AI': 1, '1': 0}
{'2': 8, 'AI': 1, '1': 1}
{'2': 6, 'AI': 1, '1': 0}
{'2': 4, 'AI': 2, '1': 0}
{'2': 1, 'AI': 2, '1': 0}
{'2': 7, 'AI': 2, '1': 1}
{'2': 1, '1': 0, 'AI': 1}
{'AI': 0, '2': 4, '1': 0}
{'AI': 2, '1': 1, '2': 4}
{'AI': 2, '1': 1, '2': 2}
{'AI': 2, '2': 2, '1': 0}
{'2': 2, '1': 0, 'AI': 4}
{'2': 0, 'AI': 5, '1': 0}
{'2': 8, '1': 1, 'AI': 2}
{'2': 1, '1': 0, 'AI': 0}
{'AI